In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - direct external IP access
#%sql postgresql://gpadmin@34.67.65.96:5432/madlib

# Greenplum Database 5.x on GCP - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# psycopg2 connection
import psycopg2 as p2
#conn = p2.connect('postgresql://fmcquillan@localhost:5432/madlib')
conn = p2.connect('postgresql://gpadmin@localhost:8000/madlib')
cur = conn.cursor()

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.17-dev, git revision: rel/v1.16-46-g77ee745, cmake configuration time: Thu Nov 14 17:59:26 UTC 2019, build type: release, build system: Linux-3.10.0-957.27.2.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


Pretty print run schedule

In [71]:
import numpy as np
from math import log, ceil

#input
max_iter = 81  # maximum iterations/epochs per configuration
eta = 3  # defines downsampling rate (default=3)

logeta = lambda x: log(x)/log(eta)
s_max = int(logeta(max_iter))  # number of unique executions of Successive Halving (minus one)
B = (s_max+1)*max_iter  # total number of iterations (without reuse) per execution of Succesive Halving (n,r)

#echo output
print ("max_iter = " + str(max_iter))
print ("eta = " + str(eta))
print ("B = " + str(s_max+1) + "*max_iter = " + str(B))

sum_leaf_n_i = 0 # count configurations at leaf nodes across all s

#### Begin Finite Horizon Hyperband outlerloop. Repeat indefinitely.
for s in reversed(range(s_max+1)):
    
    print (" ")
    print ("s=" + str(s))
    print ("n_i      r_i")
    print ("------------")
    counter = 0
    
    n = int(ceil(int(B/max_iter/(s+1))*eta**s)) # initial number of configurations
    r = max_iter*eta**(-s) # initial number of iterations to run configurations for

    #### Begin Finite Horizon Successive Halving with (n,r)
    #T = [ get_random_hyperparameter_configuration() for i in range(n) ] 
    for i in range(s+1):
        # Run each of the n_i configs for r_i iterations and keep best n_i/eta
        n_i = n*eta**(-i)
        r_i = r*eta**(i)
        
        print (str(n_i) + "     " + str (r_i))
        
        # check if leaf node for this s
        if counter == s:
            sum_leaf_n_i += n_i
        counter += 1
        
        #val_losses = [ run_then_return_val_loss(num_iters=r_i,hyperparameters=t) for t in T ]
        #T = [ T[i] for i in argsort(val_losses)[0:int( n_i/eta )] ]
    #### End Finite Horizon Successive Halving with (n,r)

print (" ")
print ("sum of configurations at leaf nodes across all s = " + str(sum_leaf_n_i))
print ("(if have more workers than this, they may not be 100% busy)")

max_iter = 81
eta = 3
B = 5*max_iter = 405
 
s=4
n_i      r_i
------------
81     1.0
27.0     3.0
9.0     9.0
3.0     27.0
1.0     81.0
 
s=3
n_i      r_i
------------
27     3.0
9.0     9.0
3.0     27.0
1.0     81.0
 
s=2
n_i      r_i
------------
9     9.0
3.0     27.0
1.0     81.0
 
s=1
n_i      r_i
------------
6     27.0
2.0     81.0
 
s=0
n_i      r_i
------------
5     81
 
sum of configurations at leaf nodes across all s = 10.0
(if have more workers than this, they may not be 100% busy)


Pretty print diagonal

In [72]:
import numpy as np
from math import log, ceil

#input
max_iter = 81  # maximum iterations/epochs per configuration
eta = 3  # defines downsampling rate (default=3)

logeta = lambda x: log(x)/log(eta)
s_max = int(logeta(max_iter))  # number of unique executions of Successive Halving (minus one)
B = (s_max+1)*max_iter  # total number of iterations (without reuse) per execution of Succesive Halving (n,r)

#echo output
print ("echo input:")
print ("max_iter = " + str(max_iter))
print ("eta = " + str(eta))
print ("s_max = " + str(s_max))
print ("B = " + str(s_max+1) + "*max_iter = " + str(B))

print (" ")
print ("initial n, r values for each s:")
initial_n_vals = {}
initial_r_vals = {}
# get hyper parameter configs for each s
for s in reversed(range(s_max+1)):
    
    n = int(ceil(int(B/max_iter/(s+1))*eta**s)) # initial number of configurations
    r = max_iter*eta**(-s) # initial number of iterations to run configurations for
    
    initial_n_vals[s] = n 
    initial_r_vals[s] = r 
    
    print ("s=" + str(s))
    print ("n=" + str(n))
    print ("r=" + str(r))
    print (" ")
    
print ("outer loop on diagonal:")
# outer loop on diagonal
for i in range(s_max+1):
    print (" ")
    print ("i=" + str(i))
    
    print ("inner loop on s desc:")
    # inner loop on s desc
    for s in range(s_max, s_max-i-1, -1):
        n_i = initial_n_vals[s]*eta**(-i+s_max-s)
        r_i = initial_r_vals[s]*eta**(i-s_max+s)
        
        print ("s=" + str(s))
        print ("n_i=" + str(n_i))
        print ("r_i=" + str(r_i))

echo input:
max_iter = 81
eta = 3
s_max = 4
B = 5*max_iter = 405
 
initial n, r values for each s:
s=4
n=81
r=1.0
 
s=3
n=27
r=3.0
 
s=2
n=9
r=9.0
 
s=1
n=6
r=27.0
 
s=0
n=5
r=81
 
outer loop on diagonal:
 
i=0
inner loop on s desc:
s=4
n_i=81
r_i=1.0
 
i=1
inner loop on s desc:
s=4
n_i=27.0
r_i=3.0
s=3
n_i=27
r_i=3.0
 
i=2
inner loop on s desc:
s=4
n_i=9.0
r_i=9.0
s=3
n_i=9.0
r_i=9.0
s=2
n_i=9
r_i=9.0
 
i=3
inner loop on s desc:
s=4
n_i=3.0
r_i=27.0
s=3
n_i=3.0
r_i=27.0
s=2
n_i=3.0
r_i=27.0
s=1
n_i=6
r_i=27.0
 
i=4
inner loop on s desc:
s=4
n_i=1.0
r_i=81.0
s=3
n_i=1.0
r_i=81.0
s=2
n_i=1.0
r_i=81.0
s=1
n_i=2.0
r_i=81.0
s=0
n_i=5
r_i=81
